# Logistic Growth with Stan

(c) 2021 Tom Roeschinger & Manuel Razo. This work is licensed under a [Creative Commons Attribution License CC-BY 4.0](https://creativecommons.org/licenses/by/4.0/). All code contained herein is licensed under an [MIT license](https://opensource.org/licenses/MIT).

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

import fit_seq

import numpy as np
import pandas as pd
import scipy.stats as st

import bokeh.io
import bokeh.plotting
import bokeh.layouts

import bebi103
import iqplot

bokeh.io.output_notebook()

/Users/tomroschinger/opt/anaconda3/lib/python3.8/site-packages/bebi103/viz.py:38: UserWarning: DataShader import failed with error "No module named 'datashader'".
Features requiring DataShader will not work and you will get exceptions.
  warnings.warn(


Loading BokehJS ...

Model

Logistic growth is used to describe the growth of bacteria in an environment with limited capacity, meaning the number of cells cannot exceed a certain threshold, called the carrying capacity $K$. Initially, growth is approximately exponential with rate $\lambda$, but then slows down once the population size gets close to the carrying capacity. In logistic growth, the change of population size $N$ over time is given by

$$
\frac{\mathrm{d}N}{\mathrm{d}t} = \lambda N\left(1-\frac{N}{K}\right).
$$

This differential equation can be exactly solved to obtain the population size as a function of time,

$$
N(t) = \frac{KN_0e^{\lambda t}}{K + N_0 \left(e^{\lambda t} - 1\right)}
$$

## Priors
#### Carrying Capacity
The carrying capacity is a positive parameter. For experiments measuring OD, do not measure ODs higher than 10, which gives us an upper bound. We can use a half-normal distribution as prior for the carrying capacity,

$$
K\sim\mathrm{HalfNormal}(0, \sigma_K),
$$

where we set $\sigma_K=2.5$ as a default value.
#### Growth Rate
For growth rate we have pretty well defined boundaries. The doubling time for cells cannot be faster than a minute (20 minutes to be more precise), and we will not be able do observe growth if the cells divide slower than once per day. This spans three orders of magnitude, so we should use a log-normal distribution for the growth rate.

$$
\log_{10}(\lambda) \sim\mathrm{Normal}(-1.5, 0.75)
$$

In [3]:
x_K = np.linspace(0, 10, 1000)
y_K = st.norm.pdf(x_K, 0, 2.5)

log_x_λ = np.linspace(-4, 1, 1000)
y_λ = st.norm.pdf(log_x_λ, -1.5, 0.75)

p_k = bokeh.plotting.figure(
    x_axis_label="Carrying Capacity K [OD600]", 
    y_axis_label="π(K)",
    frame_width=300,
    frame_height=300
)
p_λ = bokeh.plotting.figure(
    x_axis_label="Growth Rate λ[OD600/min]", 
    y_axis_label="π(λ)",
    frame_width=300,
    frame_height=300
)

p_k.line(x_K, y_K, line_width=2)
p_λ.line(10**(log_x_λ), y_λ, line_width=2)

bokeh.io.show(
    bokeh.layouts.row(
        [p_k, p_λ]
    )
)


In [4]:
?fit_seq.growth_models.logistic_growth_fit

Signature:
fit_seq.growth_models.logistic_growth_fit(
    t,
    y,
    y_0_sigma=0.01,
    K_sigma=1,
    sigma_sigma=0.001,
    log_lambda_mu=-1.5,
    log_lambda_sigma=0.75,
    chains=4,
    iter_sampling=1000,
    iter_warmup=1000,
    t_ppc=[],
    return_samples=False,
)
Docstring:
Fit a logistic growth model to a single growth curve.

Parameters
----------
t : array-like
    Time measurements.
y : array-like
    Population measurements, e.g., OD or cell count.
y_0_sigma : float, default 0.01
    Standard Deviation of half normal prior for initial measurement at t=0. Default is chosen for OD measurements.
K_sigma : float, default 5
    Standard Deviation of half normal prior for carrying capacity. Default is chosen for OD measurements.
sigma_sigma : float, default 0.01
    Standard Deviation of half normal prior for standard deviation of the normal likelihood.
log_lambda_mu : float, default -1.5
    Mean of Normal prior on the log of the growth rate.
sigma_lambda_mu : float, def

In [5]:
df = pd.read_csv("../../code/processing/growth_curves_plate_reader/20210604_r1_MG1655_319_IW_tc/output/20210604_r1_gp_per_well.csv")

In [13]:
t = df.loc[df.well == "H12", "time_min"].values
y = df.loc[df.well == "H12", "OD600"].values

In [14]:
λ, K, samples  = fit_seq.growth_models.logistic_growth_fit(t, y, return_samples=True)

INFO:cmdstanpy:found newer exe file, not recompiling
INFO:cmdstanpy:compiled model file: /Users/tomroschinger/git/fit_seq/fit_seq/stan_code/logistic_growth_model
INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:start chain 2
INFO:cmdstanpy:start chain 3
INFO:cmdstanpy:start chain 4
INFO:cmdstanpy:finish chain 4
INFO:cmdstanpy:finish chain 1
INFO:cmdstanpy:finish chain 2
INFO:cmdstanpy:finish chain 3


In [15]:
data_ppc = samples.posterior["y_predict"].stack(
            {"sample": ("chain", "draw")}
        ).transpose("sample", "y_predict_dim_0")

In [16]:
bokeh.io.show(
    bebi103.viz.corner(
        samples,
        parameters=["K", "lambda", "y_0", "sigma"]
    )
)

In [17]:
bokeh.io.show(
    bokeh.layouts.row(
        [
            bebi103.viz.predictive_regression(
                data_ppc,
                samples_x=t,
                data=np.transpose([t, y])
            ),
            bebi103.viz.predictive_regression(
                data_ppc,
                samples_x=t,
                data=np.transpose([t, y]),
                diff=True
            )
        ]
    )
)

In [18]:
data_ppc[8].values

array([0.0441464, 0.044745 , 0.0441314, 0.0443667, 0.0442033, 0.0445197,
       0.0442635, 0.0441637, 0.0445935, 0.0446748, 0.0446065, 0.044619 ,
       0.0450814, 0.0443298, 0.0449452, 0.0439734, 0.0448552, 0.0453704,
       0.0448404, 0.0445028, 0.0451825, 0.0450163, 0.0443443, 0.0448503,
       0.0448846, 0.0443068, 0.0448662, 0.0455207, 0.044854 , 0.0454525,
       0.0452552, 0.0447502, 0.0452233, 0.0452977, 0.0453904, 0.0451412,
       0.0457669, 0.0453375, 0.0452296, 0.045572 , 0.0461387, 0.0452834,
       0.0458838, 0.0458707, 0.0451177, 0.0460965, 0.0460525, 0.0457692,
       0.0457869, 0.0459766, 0.04505  , 0.0457911, 0.0454737, 0.0454204,
       0.0459374, 0.0464689, 0.0456987])

In [19]:
p = bokeh.plotting.figure()
for i in range(3100, 3102):
    p.line(t, data_ppc[i].values)

bokeh.io.show(p)